<a href="https://colab.research.google.com/github/yohoobot/works/blob/main/wyy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install selenium
!pip install webdriver-manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 25.5 MB/s eta 0:00:00


In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time

# 设置 Selenium 选项
options = Options()
options.add_argument("--headless")  # 运行无头模式（不打开浏览器）
options.add_argument("--disable-gpu")  # 适用于某些系统
options.add_argument("--window-size=1920x1080")  # 设置窗口大小
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# 启动 WebDriver（自动下载 ChromeDriver）
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 关键词列表
KEYWORDS = ["餐厅", "restaurant", "canteen", "背景音乐", "dining", "cafe", "coffee shop"]

# 存储数据
data_list = []

for keyword in KEYWORDS:
    print(f"正在搜索关键词：{keyword} ...")

    # 访问网易云音乐搜索页面
    search_url = f"https://music.163.com/#/search/m/?s={keyword}&type=1000"
    driver.get(search_url)

    # 等待页面加载
    time.sleep(5)

    # 切换到 iframe
    try:
        driver.switch_to.frame("contentFrame")
    except:
        print(f"无法切换 iframe，可能页面结构有变动（{keyword}）")
        continue

    # 获取所有歌单信息
    playlists = driver.find_elements(By.CSS_SELECTOR, ".m-cvrlst li")

    if not playlists:
        print(f"未找到任何歌单（{keyword}）")
        continue

    for playlist in playlists:
        try:
            # 获取歌单名称
            playlist_name = playlist.find_element(By.CSS_SELECTOR, ".dec a").get_attribute("title")
            # 获取歌单链接
            playlist_url = playlist.find_element(By.CSS_SELECTOR, ".dec a").get_attribute("href")
            # 获取播放次数
            play_count_text = playlist.find_element(By.CSS_SELECTOR, ".nb").text
            # 解析播放次数（转换为数字）
            if "万" in play_count_text:
                play_count = int(float(play_count_text.replace("万", "")) * 10000)
            elif "亿" in play_count_text:
                play_count = int(float(play_count_text.replace("亿", "")) * 100000000)
            else:
                play_count = int(play_count_text.replace(",", ""))

            # 存储数据
            data_list.append({
                "关键词": keyword,
                "歌单名称": playlist_name,
                "歌单链接": playlist_url,
                "播放次数": play_count
            })
        except Exception as e:
            print(f"解析歌单时出错：{e}")

    # 休息2秒，防止反爬
    time.sleep(2)

# 关闭浏览器
driver.quit()

# 存储数据
df = pd.DataFrame(data_list)

# 按播放次数排序
df = df.sort_values(by="播放次数", ascending=False)

# 保存为 CSV 文件
df.to_csv("netease_canteen_music_playlist.csv", encoding="utf-8-sig", index=False)

print("网易云音乐歌单爬取完成！")


WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x573f879a84e3 <unknown>
#1 0x573f876d7c76 <unknown>
#2 0x573f876fe757 <unknown>
#3 0x573f876fd029 <unknown>
#4 0x573f8773bccc <unknown>
#5 0x573f8773b47f <unknown>
#6 0x573f87732de3 <unknown>
#7 0x573f877082dd <unknown>
#8 0x573f8770934e <unknown>
#9 0x573f879683e4 <unknown>
#10 0x573f8796c3d7 <unknown>
#11 0x573f87976b20 <unknown>
#12 0x573f8796d023 <unknown>
#13 0x573f8793b1aa <unknown>
#14 0x573f879916b8 <unknown>
#15 0x573f87991847 <unknown>
#16 0x573f879a1243 <unknown>
#17 0x7dd8f1e10ac3 <unknown>
